In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)

In [ ]:
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

In [ ]:
def previous_applications(num_rows = None, nan_as_category = True):
    prev = pd.read_csv('../input/home-credit-default-risk/previous_application.csv', nrows = num_rows)
    
    a = ['Family', 'Spouse, partner', 'Children', 'Other_B', 'Other_A', 'Group of people']
    df["NAME_TYPE_SUITE"] = df["NAME_TYPE_SUITE"].replace(a, 'Accompanied')
    
    b = ['Auto Accessories', 'Jewelry', 'Homewares', 'Medical Supplies', 'Vehicles', 'Sport and Leisure', 
         'Gardening', 'Other', 'Office Appliances', 'Tourism', 'Medicine', 'Direct Sales', 'Fitness', 'Additional Service', 
         'Education', 'Weapon', 'Insurance', 'House Construction', 'Animals'] 
    df["NAME_GOODS_CATEGORY"] = df["NAME_GOODS_CATEGORY"].replace(b, 'others')
    
    c = ['AP+ (Cash loan)', 'Channel of corporate sales', 'Car dealer']
    df["CHANNEL_TYPE"] = df["CHANNEL_TYPE"].replace(c, 'Other_Channel')
    
    d = ['Auto technology', 'Jewelry', 'MLM partners', 'Tourism']
    df["NAME_SELLER_INDUSTRY"] = df["NAME_SELLER_INDUSTRY"].replace(d, 'Others')
    
    e = ['Refusal to name the goal', 'Money for a third person', 'Buying a garage','Gasification / water supply',
     'Hobby','Business development','Buying a holiday home / land','Furniture','Car repairs',
     'Buying a home','Wedding / gift / holiday']
    df["NAME_CASH_LOAN_PURPOSE"] = df["NAME_CASH_LOAN_PURPOSE"].replace(e, 'Other_Loan')
    
    
    prev['NFLAG_LAST_APPL_IN_DAY'] = prev['NFLAG_LAST_APPL_IN_DAY'].astype("O")
    
    
    df['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    df['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    df['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    df['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    df['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    
    
    
    df['NEW_APP_CREDIT_RATE'] = df['AMT_APPLICATION'] / df['AMT_CREDIT']
    
    df["NEW_APP_CREDIT_RATE_RATIO"] = df["NEW_APP_CREDIT_RATE"].apply(lambda x: 1 if(x<=1) else 0) # objecte çevirmek gerekebilir. 0,1 yerine evet hayır yazılabilir.
    
    df['NEW_AMT_PAYMENT_RATE'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']

    df['NEW_APP_GOODS_RATE'] = df['AMT_APPLICATION'] / df['AMT_GOODS_PRICE']

    df['NEW_CREDIT_GOODS_RATE'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']

    df['NEW_RETURN_DAY'] =  df['DAYS_DECISION'] + df['CNT_PAYMENT'] * 30

    df['NEW_DAYS_TERMINATION_DIFF'] = df['DAYS_TERMINATION'] - df['NEW_RETURN_DAY']

    df['NEW_DAYS_DUE_DIFF'] = df['DAYS_LAST_DUE_1ST_VERSION'] - df['DAYS_FIRST_DUE'] 
    
    df["NEW_CNT_PAYMENT"] = pd.cut(x=df['CNT_PAYMENT'], bins=[0, 12, 60,120], labels=["Kısa", "Orta", "Uzun"])
    
    df["NEW_END_DIFF"] = df["DAYS_TERMINATION"] - df["DAYS_LAST_DUE"]

    weekend = ["SATURDAY","SUNDAY"]                              
    df["WEEKDAY_APPR_PROCESS_START"] = df["WEEKDAY_APPR_PROCESS_START"].apply(lambda x : "WEEKEND" if (x in weekend) else "WEEKDAY")
    
    prev.drop("FLAG_LAST_APPL_PER_CONTRACT", axis = 1, inplace = True)
    


    
    prev, cat_cols = one_hot_encoder(prev, nan_as_category= True)
    
    
    num_aggregations = {
        'SK_ID_PREV' : 'count',
        'AMT_ANNUITY': ['min', 'max', 'median', 'mean'], # dağılım düzgün değil mean çıkarılmalı mı? | Revolving olan değerlerin amt annuity, application vs 0 geliyor.
        'AMT_APPLICATION': ['min', 'max', 'mean', 'median'],
        'AMT_CREDIT': ['min', 'max', 'mean', 'median'],
        #'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean', 'median'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean', 'median'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean','median'],
        'DAYS_DECISION': ['min', 'max', 'mean','median'],
        'CNT_PAYMENT': ['mean', 'sum','median','min','max'],
        #'NFLAG_LAST_APPL_IN_DAY' : ['mean'] # ohe ye sokuldu artık sonuna _0 ve _1 ekli halleri gelecek.
        'NAME_CONTRACT_TYPE' : []
        
        
    }
    # Previous applications categorical features
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']
    
    prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
    # Previous Applications: Approved Applications - only numerical features
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
    # Previous Applications: Refused Applications - only numerical features
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
    del refused, refused_agg, approved, approved_agg, prev
    gc.collect()
    return prev_agg

In [ ]:
prev = pd.read_csv('../input/home-credit-default-risk/previous_application.csv')

In [ ]:
prev["AMT_APPLICATION"].agg("min")

In [ ]:
prev[(prev["AMT_APPLICATION"] == 0) & (prev["NAME_CONTRACT_STATUS"] == "Approved") & (prev["NAME_CONTRACT_TYPE"] != "Revolving loans")]

In [ ]:
prev[(prev["AMT_ANNUITY"] == 0) & (prev["NAME_CONTRACT_TYPE"] == "Revolving loans")]

In [ ]:
prev["AMT_DOWN_PAYMENT"].agg("min")

In [ ]:
liste = ["Approved","Refused","Canceled"]

for i in prev["NAME_CONTRACT_STATUS"].unique():
    print(i)
    print(prev[(prev["FLAG_LAST_APPL_PER_CONTRACT"] == "N") & (prev["NAME_CONTRACT_STATUS"] == i)].shape[0])

In [ ]:
prev["NAME_CONTRACT_TYPE"].value_counts()

In [ ]:
for i in prev["NAME_CONTRACT_STATUS"].unique():
    print(i)

In [ ]:
prev.info()

In [ ]:
prev['NFLAG_LAST_APPL_IN_DAY'] = prev['NFLAG_LAST_APPL_IN_DAY'].astype("O")